# Names 
Spencer Bernardo-Cheng
Max Goldstein
Robbie Keehan

# Data Preparation

In [95]:
import numpy as np
import pandas as pd
datafb = pd.read_csv("./FB.csv")
datagoogle = pd.read_csv("./GOOGL.csv")
datamsft = pd.read_csv("./MSFT.csv")
dataam = pd.read_csv("./AMZN (1).csv")
datanflx = pd.read_csv("./NFLX.csv")

del datafb['Date']
del datagoogle['Date']
del datamsft['Date']
del datanflx['Date']


data = pd.concat([datafb, datagoogle, datamsft, dataam, datanflx], axis=1, sort=False)
data = data[90:]
del data['Change']
del data['Gain']
del data['Loss']
del data['Average Gain']
del data['Average Loss']
del data['RS']
del data['DM+1']
del data['DM-1']
del data['TR-14']
del data['DM+1-14']
del data['DM-1+14']
del data['dl+1-14']
del data['dl-1-14']
del data['dl14diff']
del data['dl14sum']
del data['DX']
del data['TR']

up = .03
down =-.03
ydata = data['% Change']
ydata = ydata.apply(lambda x:2 if x>=up else (1 if (x< up and x > down) else 0))
ydata = pd.DataFrame(ydata)
ydata['newy'] = ""
# created target data in format to match output of multiple timesteps of 14 days ahead
# [0,1,2,3,4,5,6,7]
# [1,2,3,4,5,6,7,8]
for index, row in ydata.iterrows():
    temp = np.array(ydata['% Change'][index-90:index-90+14].values)
    ydata['newy'].at[index] = temp
    
# removed last 14 because their targets are not able to created
ydata = ydata[0:-13]
data = data[0:-13]
del ydata['% Change']
print(ydata.shape)
print(ydata)


(1789, 1)
                                            newy
90    [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2]
91    [0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0]
92    [0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0]
93    [0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2]
94    [2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2]
...                                          ...
1874  [2, 0, 2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0]
1875  [0, 2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0]
1876  [2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0]
1877  [0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0]
1878  [2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2]

[1789 rows x 1 columns]


# One Hot Encoding The Target Data

In [96]:
from keras.utils.np_utils import to_categorical
print(ydata.head())
ydata['newy'] = ydata['newy'].apply(lambda x: to_categorical(x,3))
print(ydata)

                                          newy
90  [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2]
91  [0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0]
92  [0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0]
93  [0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2]
94  [2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2]
                                                   newy
90    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 0.0, ...
91    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 0.0, ...
92    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
93    [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, ...
94    [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
...                                                 ...
1874  [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
1875  [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, ...
1876  [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
1877  [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, ...
1878  [[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, ...

[1789 rows x 1 columns]


# Min-Max Scaling the Time Series

In [72]:
from sklearn import preprocessing
del data['Date']

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data.iloc[:,:].values)
print(x_scaled)


[[0.00706588 0.00746643 0.00598557 ... 0.00188326 0.00188326 0.42421594]
 [0.00472753 0.00440971 0.00332533 ... 0.00187282 0.00187282 0.19557866]
 [0.00447336 0.00450992 0.00414387 ... 0.00203991 0.00203991 0.10745063]
 ...
 [0.88415002 0.87677893 0.88325571 ... 0.68480541 0.68480541 0.02587954]
 [0.89091091 0.88314295 0.89312939 ... 0.69798828 0.69798828 0.02485125]
 [0.89243593 0.89982967 0.89144114 ... 0.71636145 0.71636145 0.03687678]]


# Utilizing Time Series Generator

In [100]:
from keras.models import Sequential
from keras.layers import Dense
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 2
series = np.array(x_scaled)
target = np.array(ydata)
generator = TimeseriesGenerator(series, target, length=90, batch_size=1)
xtime = []
ytarget =[]
for i in range(len(generator)):
    x, y = generator[i]
    xtime.append(x[0])
    ytarget.append(y[0])
xtime = np.array(xtime)
ytarget = np.array(ytarget)



# Time Series Fold Splits

In [101]:
from sklearn.model_selection import train_test_split
import keras.utils
from keras import utils as np_utils
x_train,x_test,y_train,y_test = train_test_split(xtime, ytarget, test_size = 1/5, random_state = 1)

print(x_train.shape)
print(y_train)


(1359, 90, 36)
[[array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)]
 [array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)]
 [array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)]
 ...
 [array([[0., 0., 1.],
       [1., 0., 0.

In [102]:

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(90, 36), return_sequences=True))
model.add(Lambda(lambda x: x[:, -14:, :]))
model.add(Dense(3,activation='sigmoid'))

model.summary()
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(x_train, y_train, validation_data=(x_test,y_test), shuffle=True, epochs=100, verbose=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 90, 100)           54800     
_________________________________________________________________
lambda_1 (Lambda)            (None, 14, 100)           0         
_________________________________________________________________
dense_13 (Dense)             (None, 14, 3)             303       
Total params: 55,103
Trainable params: 55,103
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_13 to have 3 dimensions, but got array with shape (1359, 1)